**Importer les bibliothèques nécessaires**

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

***On lit le fichier CSV contenant les données nettoyées.***

In [4]:
# Lecture du fichier nettoyé
df = pd.read_csv("data/avito_cars_clean.csv")

# Vérification
df.head()

,annee,boite,carburant,kilometrage,marque,modele,nombre_portres,premiere_main,puissance_fiscale,etat,prix
0,2023,Automatique,Essence,22499.5,Alfa Romeo,Tonale,5,Oui,8,Excellent,306000
1,2018,Manuelle,Diesel,104999.5,Dacia,Duster,5,Oui,6,Excellent,155000
2,2023,Automatique,Diesel,67499.5,Hyundai,Accent,5,Oui,6,Excellent,179000
3,2023,Automatique,Diesel,22499.5,Peugeot,5008,5,Oui,8,Excellent,349000
4,2021,Automatique,Diesel,77499.5,Kia,Sportage,5,Oui,12,Très bon,685000


***Vérifier et nettoyer les données***

In [5]:
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68415 entries, 0 to 68414
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   annee              68415 non-null  int64  
 1   boite              68415 non-null  object 
 2   carburant          68415 non-null  object 
 3   kilometrage        68415 non-null  float64
 4   marque             68415 non-null  object 
 5   modele             68415 non-null  object 
 6   nombre_portres     68415 non-null  int64  
 7   premiere_main      68415 non-null  object 
 8   puissance_fiscale  68415 non-null  int64  
 9   etat               68415 non-null  object 
 10  prix               68415 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 5.7+ MB
None
annee                0
boite                0
carburant            0
kilometrage          0
marque               0
modele               0
nombre_portres       0
premiere_main        0
puissance_fiscal

**Cela signifie que les données sont bien structurées et prêtes pour l'analyse, et qu’il n’y a pas de données manquantes à traiter.**

**XGBoost ne gère pas les textes directement, donc on convertit les colonnes textuelles en chiffres.**

In [6]:
label_cols = ['boite', 'carburant', 'marque', 'modele', 'etat']
label_encoders = {}

for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


**On définit les colonnes d’entrée et la colonne cible (le prix).**

In [7]:
X = df.drop(columns=['prix'])
y = df['prix']


**On va entraîner le modèle sur 80% des données et tester sur 20%.**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Utilise l'encodeur LabelEncoder de sklearn pour convertir les colonnes catégorielles en valeurs numériques.**

In [9]:
from sklearn.preprocessing import LabelEncoder

# Créer un encodeur
label_encoder = LabelEncoder()

# Encoder les colonnes catégorielles
categorical_columns = ['boite', 'carburant', 'marque', 'modele', 'premiere_main', 'etat']

# Appliquer l'encodeur à chaque colonne catégorielle
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Vérifier les premières lignes après l'encodage
df.head()


,annee,boite,carburant,kilometrage,marque,modele,nombre_portres,premiere_main,puissance_fiscale,etat,prix
0,2023,0,2,22499.5,2,705,5,1,8,3,306000
1,2018,1,0,104999.5,18,292,5,1,6,3,155000
2,2023,0,0,67499.5,34,106,5,1,6,3,179000
3,2023,0,0,22499.5,56,72,5,1,8,3,349000
4,2021,0,0,77499.5,40,656,5,1,12,6,685000


On selectionne les colonnes comme caractéristiques et la colonne prix comme la cible.

In [10]:
# Définir les caractéristiques (X) et la cible (y)
X = df.drop(columns=['prix'])  # Supprimer la colonne cible
y = df['prix']  # La cible


***On divise les données en ensembles d'entraînement et de test***

In [11]:
from sklearn.model_selection import train_test_split

# Diviser les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


***Création et entrainement du chatbot***

In [12]:
import xgboost as xgb

# Créer le modèle XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)

# Entraîner le modèle
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

***On passe à l'etape des prédictions et d'évaluation des performances***

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Prédictions
y_pred = model.predict(X_test)

# Calcul des métriques d'évaluation
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Calcul de la racine carrée manuellement
r2 = r2_score(y_test, y_pred)

# Affichage des résultats
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R²: {r2}")


Mean Absolute Error (MAE): 24859.77509857693
Root Mean Squared Error (RMSE): 53365.31902268839
R²: 0.7730891260508274
